# HR ATTRITION ANALYSIS

Employee attrition occurs when an employee leaves a company of his/her own choice, and usually results in a vacancy that needs to be filled quickly. This can cost the company aconsiderable amount of money, especailly if the employee who left had been hired recently. Imagine the cost to the company if they had to hire two times in a year to fill the same position! By proactively identifying employees who are more likely to quit, the company can incentivize them to stay, saving the company from expensive hiring costs. **In this analysis, I will explore the variables that affect an employee's tendency to leave the company. I will then offer data-supported recommendations to minimize turnover.**

This report will be broken down into 5 sections:
 - Part 1: Load and explore the dataset. In this portion, I begin by loading the packages we will need in our analysis, loading the data and running some preliminary explorations of the dataset, including basic statistics for each variable.
 - Part 2: Graph the dataset. In this portion, I will use graphs to answer five questions from management.
 - Part 3: Prepare data for modeling. In this portion, I split the data into two parts, a training part that I will use to create the model, and a test part that I will use to make sure that my model is versatile enough to be applied to other datasets.
 - Part 4: Build and evaluate a model to predict attrition. In this portion, I will use two different techniques, logistic regression and descision trees, to predict attrition.
 - Part 5: Recommendations. In this section, I will offer my reccomendations to minimize attrition based on my discoveries.
 
 ---
 **Note:** This analysis only attempts to discover if any correlation exists between attrition and other variables. That is, I am trying to see if I can predict an employee's likelihood to quit based on information about that employee. This analysis cannot be used to determine that any of these variables cause attrition. Identifying causitive factors that affect an employee's likelihood to quit are beyond the scope of this report.

## Part 1: Load and explore the dataset.

In this portion, I begin by loading the packages we will need in our analysis, loading the data and running some preliminary explorations of the dataset, including basic statistics for each variable.

In [92]:
# Load the appropriate libraries and suppress warning messages.
install.packages('ggcorrplot')
shh <- suppressPackageStartupMessages
shh(library(ggplot2))
shh(library(caret))
shh(library(ggcorrplot))
shh(library(pROC))
shh(library(rpart))

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [93]:
# Load the CSV file with employee data.
hr_data <- read.csv('http://bit.ly/attrition_project')

In [94]:
# Calculate the dimensions of the dataset and list all columns/data types.
str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : int  1 0 1 0 0 0 0 0 0 0 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 ...
 $ DistanceFromHome   

In [95]:
# Preview the first few records.
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


In [96]:
# Generate some descriptive stats.
summary(hr_data)

   Attrition      EmployeeNumber        Age           Gender     MaritalStatus
 Min.   :0.0000   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 1st Qu.:0.0000   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
 Median :0.0000   Median :1020.5   Median :36.00                Single  :470  
 Mean   :0.1612   Mean   :1024.9   Mean   :36.92                              
 3rd Qu.:0.0000   3rd Qu.:1555.8   3rd Qu.:43.00                              
 Max.   :1.0000   Max.   :2068.0   Max.   :60.00                              
                                                                              
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :

## Part 2: Graph the dataset.

In this portion, I will use graphs to answer five questions from management.

### Question 1: Does one gender tend to quit more frequently than the other?

This is a great question that could say a lot about the way the company treats its employees. One gender tending to quit more frequently than the other could imply a discrimination problem in the company. To answer this question, I will use graphs to visualize the employees who quit based on gender. In the first graph, the number of males leaving the company is about twice the number of females leaving the company. However, this graph could be deceiving because number of male employees is larger than the number of female employees. To see if one gender truly does tend to quit more frequently than the other the proportion of males and the proportion of females who quit must be considered. This second graph clearly shows that the percentage of males who quit insignificantly differs from the proportion of females who quit. Thus, the data doesn't support the idea that one gender tends to quit more frequently than the other.

In [97]:
# Create impactful, colorful graphs to help answer question 1.
ggplot(hr_data, aes(Gender, fill = factor(Attrition))) +
  geom_bar() +
  ggtitle("Attrition Count by Gender") +
  xlab("Gender") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(Gender, fill = factor(Attrition))) +
  geom_bar(position = "fill") +
  ggtitle("Attrition Percentage by Gender") +
  xlab("Gender") +
  ylab("% of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))

### Question 2: Does age appear to make a difference? Are our younger workers less committed to our company?

If younger workers are less committed to the company, the costs of replacing them could quickly add up. To see if age makes a difference, I will use graphs to visualize the ages of employees who quit. The first graph shows that employees around 30 years old are leaving the company more than employees of other ages. However, similar to the the previous question, this graph can be deceiving, as the majority of employees are in that age range. In the second graph, we instead consider the percentage of employees who leave when compared to all employees with a similar age. This graph does show there is a larger percentage of employees under the age of 30 who quit compared to employees that are older. Does this imply that younger workers are less committed to the company? I disagree. The third graph shows the monthly income of each employee by his/her age. The graph shows that younger employees have a significantly lower monthly income than other employees which is a factor to be considered when asking why younger workers are leaving the company. Although another analysis could be conducted to better predict why young employees are quitting, we cannot simply assume that they are leaving due to a lack of commitment in this report.

In [98]:
# Create impactful, colorful graphs to help answer question 2.
ggplot(hr_data, aes(Age, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 2) +
  ggtitle("Attrition Count by Age") +
  xlab("Age") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(Age, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 2, position = "fill") +
  ggtitle("Attrition Percentage by Age") +
  xlab("Age") +
  ylab("% of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(Age, MonthlyIncome)) +
  geom_jitter(color = "blue") +
  ggtitle("Age vs. Monthly Income") +
  xlab("Age") +
  ylab("Monthly Income")

### Question 3: Are we working our employees too hard? Is our overtime requirement for some employees driving them away?

Over working employees is a real concern, especially if that is driving them away. A simple graph can help show if a greater number of employees leaving were also working over time. In answering this question, I will skip straight to the percentage graph to see if a greater percentage of attrition by employees who work over time when compared to the percentage of attrition by employees who didn't work over time. This graph shows that there is definitely a larger percentage of employees who leave if they are working over time. Again, this analysis cannot say that over time work is causing them to quit, but there is a clear connection between over time work and employees who leave the company.

In [112]:
# Create impactful, colorful graphs to help answer question 3.
ggplot(hr_data, aes(OverTime, fill = factor(Attrition))) +
  geom_bar(position = "fill") +
  ggtitle("Attrition Percentage for Over Time Work") +
  xlab("Over Time?") +
  ylab("% of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))

### Question 4: Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?



In [100]:
# Create impactful, colorful graphs to help answer question 4.
ggplot(hr_data, aes(MonthlyIncome, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 1000) +
  ggtitle("Attrition Count by Monthly Income") +
  xlab("Monthly Income") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(MonthlyIncome, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 1000, position = "fill") +
  ggtitle("Attrition Percentage by Monthly Income") +
  xlab("Monthly Income") +
  ylab("% of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(PercentSalaryHike, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 1) +
  ggtitle("Attrition Count by % Salary Hike") +
  xlab("% Salary Hike") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(PercentSalaryHike, fill = factor(Attrition))) +
  geom_area(stat = "bin", binwidth = 1, position = "fill") +
  ggtitle("Attrition Percentage by % Salary Hike") +
  xlab("% Salary Hike") +
  ylab("% of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))

### Question 5: Our concern is that the highest performing employees are leaving at a higher rate. Is that true?



In [101]:
# Create impactful, colorful graphs to help answer question 5.
ggplot(hr_data, aes(factor(PerformanceRating), fill = factor(Attrition))) +
  geom_bar() +
  ggtitle("Attrition Count by Performance Rating") +
  xlab("Performance Rating") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))
ggplot(hr_data, aes(factor(PerformanceRating), fill = factor(Attrition))) +
  geom_bar(position = "fill") +
  ggtitle("Attrition Count by Performance Rating") +
  xlab("Performance Rating") +
  ylab("# of Employees") +
  labs(fill = "Quit?") +
  scale_fill_manual(name = "Quit?", labels = c("No", "Yes"),
                    values = c("gray", "red"))

## Part 3: Prepare data for modeling.

In this portion, I split the data into two parts--a training part that I will use to create the model, and a test part that I will use to make sure that my model is versatile enough to be applied to other datasets. Because the model will be sensitive to missing values in the dataset, before I split the data I first determine if any null values exist. With no missing values, I can continue. I will use 80% of our data to train the model, and then use the remaining 20% to test the effectiveness of the model.

In [102]:
# Determine if any missing/null values exist.
sum(is.na(hr_data))

[1] 0

In [103]:
# Split the data into training and test sets (using a set.seed of "3456").
set.seed(3456)
trainIndex <- createDataPartition(hr_data$Attrition, p = .8, list = FALSE, times = 1)
hr_train <- hr_data[trainIndex,]
hr_test <- hr_data[-trainIndex,]
head(hr_train)
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
2,0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
3,1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
4,0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
5,0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
7,0,10,59,Female,Married,Research & Development,Laboratory Technician,2670,Yes,Medical,...,4,20,3,3,12,4,1,0,0,0


,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,4,22,1,2,1,1,1,0,0,0
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,3,13,1,5,6,0,5,4,0,3
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,3,12,2,5,7,0,6,2,0,5
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,4,23,0,4,10,7,5,3,0,3
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,3,11,0,4,13,0,12,6,2,11


## Part 4: Regression Modeling Analysis

Now that the data is split, I create a prediction model using general linearization. With this model, I can evaluate the statistical significance of each variable in the dataset. The most significant variables are whether the employee worked overtime, traveled frequently, had a long commute, felt involved and satisfied with his/her job, had worked for multiple companies, and had been promoted recently. I will revisit some of these variables when I offer my reccomendations. I will wrap up my regression analysis by adding the predictions made by the model to the test data and saving a simplified CSV file containing only the employee's ID, actual attrition, and prediction attrition.

### Logistic Regression


In [104]:
# Use glm to create a prediction model.
hr_model_lr <- glm(Attrition ~ ., data = hr_train, family = binomial)
summary(hr_model_lr)


Call:
glm(formula = Attrition ~ ., family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7806  -0.4957  -0.2701  -0.0983   3.5749  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.288e+01  4.547e+02  -0.028 0.977405    
EmployeeNumber                   -2.184e-04  1.666e-04  -1.311 0.189827    
Age                              -3.916e-02  1.489e-02  -2.629 0.008560 ** 
GenderMale                        4.978e-01  2.084e-01   2.388 0.016930 *  
MaritalStatusMarried              4.735e-01  2.996e-01   1.580 0.114028    
MaritalStatusSingle               1.127e+00  3.837e-01   2.936 0.003322 ** 
DepartmentResearch & Development  1.277e+01  4.547e+02   0.028 0.977603    
DepartmentSales                   1.258e+01  4.547e+02   0.028 0.977922    
JobRoleHuman Resources            1.301e+01  4.547e+02   0.029 0.977178    
JobRoleLaboratory Technician      1.314

In [113]:
# Perfect model by taking out non-significant variables.
hr_model_lr <- glm(Attrition ~ Age +
                   OverTime +
                   JobInvolvement +
                   JobSatisfaction +
                   YearsInCurrentRole +
                   YearsSinceLastPromotion,
                   data = hr_train, family = binomial)
summary(hr_model_lr)


Call:
glm(formula = Attrition ~ Age + OverTime + JobInvolvement + JobSatisfaction + 
    YearsInCurrentRole + YearsSinceLastPromotion, family = binomial, 
    data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5653  -0.5743  -0.3804  -0.2371   2.9273  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              2.29651    0.52773   4.352 1.35e-05 ***
Age                     -0.05916    0.01079  -5.482 4.21e-08 ***
OverTimeYes              1.51830    0.17888   8.488  < 2e-16 ***
JobInvolvement          -0.48034    0.12032  -3.992 6.55e-05 ***
JobSatisfaction         -0.28959    0.07918  -3.658 0.000255 ***
YearsInCurrentRole      -0.19499    0.03614  -5.395 6.85e-08 ***
YearsSinceLastPromotion  0.11735    0.03431   3.420 0.000625 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1023.56  on 1175  degrees of 

In [106]:
# Evaluate variable importance.
varImp(hr_model_lr)

,Overall
,<dbl>
Age,5.477135
OverTimeYes,8.381717
BusinessTravelTravel_Frequently,3.287893
BusinessTravelTravel_Rarely,1.772800
JobInvolvement,4.218457
JobSatisfaction,3.756813
YearsInCurrentRole,5.520114
YearsSinceLastPromotion,3.352115


In [114]:
# Use the model to predict attrition in the test data.
hr_test$AttritionPredictionLR <- predict(hr_model_lr, newdata = hr_test,
                                       type = "response")
print(head(hr_test[c("Attrition", "AttritionPredictionLR")]))

   Attrition AttritionPredictionLR
6          0            0.03883829
8          0            0.14332241
11         0            0.04502588
17         0            0.27513945
22         1            0.15864617
23         0            0.06475354


In [115]:
# Create a ROC curve.
ROC <- roc(hr_test$Attrition, hr_test$AttritionPredictionLR)

# Plot the ROC curve.
plot(ROC, col = "blue")

# Calculate the area under the curve.
auc(ROC)

Setting levels: control = 0, case = 1
Setting direction: controls < cases


Area under the curve: 0.7344

### Decision Tree

In [109]:
hr_model_dt <- rpart(Attrition ~ ., data = hr_train, method = "class")
print(hr_model_dt)
summary(hr_model_dt)

n= 1176 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 1176 185 0 (0.84268707 0.15731293)  
   2) OverTime=No 835  82 0 (0.90179641 0.09820359)  
     4) TotalWorkingYears>=2.5 763  59 0 (0.92267366 0.07732634) *
     5) TotalWorkingYears< 2.5 72  23 0 (0.68055556 0.31944444)  
      10) JobRole=Research Scientist 22   1 0 (0.95454545 0.04545455) *
      11) JobRole=Human Resources,Laboratory Technician,Sales Representative 50  22 0 (0.56000000 0.44000000)  
        22) Age>=32.5 8   0 0 (1.00000000 0.00000000) *
        23) Age< 32.5 42  20 1 (0.47619048 0.52380952)  
          46) MonthlyIncome>=2633.5 17   5 0 (0.70588235 0.29411765) *
          47) MonthlyIncome< 2633.5 25   8 1 (0.32000000 0.68000000) *
   3) OverTime=Yes 341 103 0 (0.69794721 0.30205279)  
     6) MonthlyIncome>=2537 278  60 0 (0.78417266 0.21582734)  
      12) JobRole=Healthcare Representative,Manager,Manufacturing Director,Research Director,Research Scientist 156  17 0 (0.891025

In [118]:
# Perfect model by taking out non-significant variables.
hr_model_dt <- prune(hr_model_dt, cp =
                     hr_model_dt$cptable[which.min(hr_model_dt$cptable[,"xerror"]),"CP"])
summary(hr_model_dt)

Call:
rpart(formula = Attrition ~ ., data = hr_train, method = "class")
  n= 1176 

          CP nsplit rel error    xerror       xstd
1 0.06216216      0 1.0000000 1.0000000 0.06749122
2 0.02972973      2 0.8756757 1.0000000 0.06749122
3 0.02702703      4 0.8162162 0.9945946 0.06734252
4 0.01216216      6 0.7621622 0.9405405 0.06581624
5 0.01000000     10 0.7135135 0.8918919 0.06437869

Variable importance
        MonthlyIncome              OverTime               JobRole 
                   21                    14                    10 
    TotalWorkingYears         MaritalStatus                   Age 
                    9                     7                     6 
   YearsInCurrentRole      StockOptionLevel        YearsAtCompany 
                    6                     6                     6 
           Department        EducationField  YearsWithCurrManager 
                    4                     3                     3 
       EmployeeNumber              JobLevel     Perce

In [119]:
# Evaluate variable importance.
varImp(hr_model_dt)

,Overall
,<dbl>
Age,31.845137
BusinessTravel,4.085723
Department,4.791697
DistanceFromHome,13.702180
EmployeeNumber,10.028778
JobLevel,21.473673
JobRole,34.244904
JobSatisfaction,6.355023
MaritalStatus,22.082996


In [128]:
# Use the model to predict attrition in the test data.
hr_test$AttritionPredictionDT <- predict(hr_model_dt, newdata = hr_test,
                                       type = "class")
print(head(hr_test[c("Attrition", "AttritionPredictionDT")]))

   Attrition AttritionPredictionDT
6          0                     0
8          0                     0
11         0                     0
17         0                     0
22         1                     0
23         0                     0


In [111]:
# Make a new dataset with only desired columns and save as CSV file.
hr_test_predict <- hr_test[c("EmployeeNumber", "Attrition", "AttritionPredictionLR")]
write.csv(hr_test_predict, "attrition_predictions.csv", row.names = FALSE)

## Recommendations

Based on this analysis, I propose four recommendations to help minimizie turnover.
1. In order to reduce the number of years since an employee's last promotion, we could have smaller, annual promotions for employees so they feel like they're being recognized for staying with the company and would thus be more inclined to stay.
2. Focus on employees who have previously worked at multiple companies. In the past, employees who had previously worked at multiple companies left, so give them more attention and make sure their needs are being met.
3. Reach out to employees who are having difficulties feeling satisfied or involved at work. Adding stressful assignments to employees who are already feeling less fulfilled in their work environment could drive them away.
4. Try to limit overtime and business related travel. When such work is necessary, give those assignments to employees who volunteer or are generally more satisfied with their jobs, which will make sure that those struggling employees aren't over worked.